In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
import requests
from io import StringIO
from sklearn.svm import SVC

In [ ]:
dataset1_url = "https://raw.githubusercontent.com/hbedros/data622-assignment4/main/data/dataset1.csv"
dataset2_url = "https://raw.githubusercontent.com/hbedros/data622-assignment4/main/data/dataset2.csv"

response1 = requests.get(dataset1_url, verify=False)
response2 = requests.get(dataset2_url, verify=False)

data1 = StringIO(response1.text)
data2 = StringIO(response2.text)
dataset1 = pd.read_csv(data1)
dataset2 = pd.read_csv(data2)

print(dataset1.head())
print(dataset2.head())


NameError: name 'requests' is not defined